#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 05 - Part 05 - NVIDIA Containers for CUDA</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

```sh
$ docker run --gpus all nvidia/cuda nvidia-smi
```
```
Tue Jun  2 11:42:20 2020
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980     Off  | 00000000:01:00.0  On |                  N/A |
|  0%   38C    P8    20W / 195W |     85MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:02:00.0 Off |                  N/A |
| 23%   30C    P8     9W / 250W |     12MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
```

* Docker with API version 1.40 supports GPUs by default, versions earlier required complex setup to work with Nvidia containers
* `--gpus all` states the container should use all GPUs available, these can be specified individually instead, eg. select GPU 1: `--gpus \"device=1\"`
* Images are provided for various versions of CUDA cuDNN, and base systems such as Ubuntu
* Let us create a simple Pytorch image to test this:

```dockerfile
FROM nvidia/cuda:10.1-cudnn7-runtime-ubuntu18.04

RUN apt-get update --fix-missing
RUN apt-get install python3 python3-pip -y
RUN pip3 install torch==1.5.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

CMD ["python3", "-c", "import torch; print(torch.__version__, torch.cuda.device_count())"]
```

* This will take a while to download Pytorch:

```sh
$ docker build -t hello-gpu .
```

* Then run:

```sh
$ docker run --gpus all -ti --rm hello-gpu
1.5.0+cu101 2
```

* This installs Python 3 (explicitly as package `python3` because of Ubuntu's naming scheme) and Pytorch
* Installing other libraries obviously done through `pip3`
* Alternative is to download `miniconda` in the image and install things with `conda`

* Resources other than the GPU can be controlled through `docker run`:
  * `-m` sets memory limit, eg. `-m 1G` for 1GB of memory max
  * `--memory-swap` sets swap file usage, -1 for no swap
  * `--cpus` sets CPU resource allocation, eg. `--cpus=\"1.5\"` allocates 1 CPU and 0.5 of another
  * `--cpuset-cpus` sets which CPU sets a container can use, eg. `--cpuset-cpus=\"1-3\"`
  * `--cpu-shares` sets the share of CPU resources out of 1024, used to balance allocation between containers
* Further flags permit more fine-grained control